In [1]:
# one off operation
!pip install duckdb --upgrade


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\mdjouallah\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Register an App if running outside Fabric to get a token
https://fivetran.com/docs/destinations/onelake/setup-guide

In [2]:
import duckdb

In [3]:
duckdb.sql(" install azure ;force install delta from core_nightly ; ")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [4]:
try:
    token = notebookutils.credentials.getToken('storage')
except:
    !pip install -q azure-identity
    from   azure.identity   import ClientSecretCredential
    import configparser
    config = configparser.ConfigParser()
    config.read("C:/KV/variable.ini")
    credential = ClientSecretCredential(
                    client_id     =config.get("myvars", "appId"),
                    client_secret =config.get("myvars", "secret") , 
                    tenant_id     =config.get("myvars", "tenantId")
                    )
    token =       credential.get_token("https://storage.azure.com/.default").token


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\mdjouallah\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [5]:
def attach_lakehouse(workspace,lakehouse,token,cache):
  con = duckdb.connect('./db.duckdb')
  con.sql(f"""          
    load azure ;
    CREATE or replace SECRET secret4 ( TYPE AZURE, PROVIDER ACCESS_TOKEN, ACCESS_TOKEN '{token}');
    set enable_object_cache = true 
     """)
  tables = con.sql(f""" SELECT distinct(split_part(file, '/', 7)) as tables FROM glob
                      ("abfss://{workspace}@onelake.dfs.fabric.microsoft.com/{lakehouse}.Lakehouse/Tables/*/_delta_log/*.json")
                    """).df()['tables'].tolist()
  for tbl in tables:
    print(tbl)
    if cache=='yes':
       con.sql(f""" drop view if exists {tbl} """)
       con.sql(f""" create or replace table {tbl} as FROM delta_scan("abfss://{workspace}@onelake.dfs.fabric.microsoft.com/{lakehouse}.Lakehouse/Tables/{tbl}")  """)
    else :
       con.sql(f""" drop table if exists {tbl} """)
       con.sql(f""" create or replace view {tbl} as FROM delta_scan("abfss://{workspace}@onelake.dfs.fabric.microsoft.com/{lakehouse}.Lakehouse/Tables/{tbl}")  """)
 

In [6]:
%%time
# use Workspace and lakehouse name, token either using notebookutils
# inside Fabric or register an App for access outside Fabric
# cache data by default
attach_lakehouse("aemodev","storage",token,cache ='yes')

states
mstdatetime


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

price


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

scada


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

duid
calendar
CPU times: total: 46.1 s
Wall time: 1min 14s


In [7]:
con = duckdb.connect('./db.duckdb')

In [8]:
con.sql("""  from scada limit 10 """).show()

┌─────────┬─────────┬──────────────────────────┬────────┬────────────┬──────────────┬──────────────┬───────────┬───────────┬──────────────┬──────────────┬────────────┬───────────┬────────────┬───────────┬───────────┬────────────┬───────────┬───────────────────┬────────────────────┬───────────────────┬───────────────┬─────────────────────┬──────────────────────┬─────────────────────┬─────────────────┬──────────┬──────────┬──────────────┬────────────────┬─────────────────┬────────────────┬───────────────┬────────────────┬─────────────────┬────────────────┬───────────────┬──────────────────────┬───────────────────────┬───────────────────────┬──────────────────────┬───────────────────────┬───────────────────────┬─────────────────────────────┬──────────────────────────────┬─────────────────────────────┬────────────────────────────┬─────────────────────────────┬──────────────────────────────┬─────────────────────────────┬────────────────────────────┬──────────────────────────────────────┬───

In [9]:
%%time
con.sql(""" 
           pivot (
            select Region, date, sum(INITIALMW)/12 as MWH from scada s
            join duid d
            on s.DUID = d.DUID
             where date > today() - 7
            GROUP BY ALL
             ) on date using sum(MWH)
           """).show()

┌─────────┬────────────────────┬────────────────────┬────────────────────┬────────────────────┬────────────────────┬────────────────────┬────────────────────┐
│ Region  │     2024-09-09     │     2024-09-10     │     2024-09-11     │     2024-09-12     │     2024-09-13     │     2024-09-14     │     2024-09-15     │
│ varchar │       double       │       double       │       double       │       double       │       double       │       double       │       double       │
├─────────┼────────────────────┼────────────────────┼────────────────────┼────────────────────┼────────────────────┼────────────────────┼────────────────────┤
│ SA1     │ 24490.753072499952 │ 21971.872856666738 │ 29221.073274166643 │ 29180.256335833215 │ 27173.628209999966 │  30906.70537666658 │  7905.703020833337 │
│ VIC1    │   133683.054553333 │ 128234.41151249962 │ 132715.00896833275 │ 127268.06821833283 │ 136407.27749333333 │ 136829.17933083358 │ 21560.823426666666 │
│ WA1     │ 3842.9460948289775 │   7249.150110

In [10]:
%%time
con.sql("""  select count(*) from scada   """).show()


┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│     28052010 │
└──────────────┘

CPU times: total: 46.9 ms
Wall time: 10.4 ms
